In [2]:
import requests
import pandas as pd
import time

TOKEN = "129ac2e3-e8a6-72c7-58c1-acced5a601bd"

# Diccionario de Exportaciones Trimestrales
indicadores_exportaciones = {
    "629659": "Total",
    "696790": "Agricultura",
    "696791": "Cría y explotación de animales",
    "697788": "Pesca, caza y captura",
    "629660": "Extracción de petroleo y gas",
    "629661": "Minería de minerales (excepto petroleo y gas)",
    "629662": "Alimentos",
    "629663": "Bebidas y tabaco",
    "629664": "Insumos y acabados textiles",
    "629665": "Productos textiles (excepto prendas de vestir)",
    "629666": "Prendas de vestir",
    "629667": "Curtido, acabado y productos de cuero y piel",
    "629668": "Madera",
    "629669": "Papel",
    "629670": "Impresión",
    "629671": "Productos derivados del petróleo y del carbón",
    "629672": "Química",
    "629673": "Plástico y hule",
    "629674": "Productos a base de minerales no metálicos",
    "629675": "Metálicas básicas",
    "629676": "Productos metálicos",
    "629677": "Maquinaría y equipo",
    "629678": "Equipo de computación, componentes y accesorios electrónicos",
    "629679": "Accesorios, aparatos y equipo de generación eléctrica",
    "629680": "Equipo de transporte",
    "629681": "Muebles, colchones y persianas",
    "629682": "Otras manufacturas",
    "629683": "No especificado"
}

resultados_totales = []
consultas_fallidas = []

def extraer_datos(ind_clave, ind_nombre, clave_estado, url):
    for intento in range(3):
        try:
            response = requests.get(url, timeout=15)
            if response.status_code == 200:
                data = response.json()
                if 'Series' in data and len(data['Series']) > 0:
                    serie = data['Series'][0]
                    if 'OBSERVATIONS' in serie:
                        for obs in serie['OBSERVATIONS']:
                            registro = {
                                'CLAVE_INDICADOR': ind_clave,
                                'SECTOR': ind_nombre,
                                'CVE_ENT': obs.get('COBER_GEO', clave_estado),
                                'TIME_PERIOD': obs.get('TIME_PERIOD'),
                                'OBS_VALUE': obs.get('OBS_VALUE')
                            }
                            resultados_totales.append(registro)
                return True
            else:
                print(f"  ⚠️ Error HTTP {response.status_code} (Intento {intento+1}/3)")
                time.sleep(2)
        except Exception as e:
            print(f"  ⚠️ Fallo de red: {type(e).__name__} (Intento {intento+1}/3)")
            time.sleep(2)
    return False

print("--- INICIANDO EXTRACCIÓN HISTÓRICA MASIVA DE EXPORTACIONES ---")

# Bucle Principal
for ind_clave, ind_nombre in indicadores_exportaciones.items():
    print(f"Consultando: {ind_clave} - {ind_nombre}")
    
    # Estados 01 a 32
    for i in range(1, 33):
        clave_estado = f"{i:02d}"
        url = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{ind_clave}/es/{clave_estado}/false/BIE-BISE/2.0/{TOKEN}?type=json"
        
        exito = extraer_datos(ind_clave, ind_nombre, clave_estado, url)
        
        if not exito:
            print(f"  ❌ Fallo total en {ind_clave}-{clave_estado}. Se envía a la cola final.")
            consultas_fallidas.append((ind_clave, ind_nombre, clave_estado, url))
            
        time.sleep(0.3)

# Reintentos de fallas
if consultas_fallidas:
    print("\n--- INICIANDO REINTENTOS DE CONSULTAS FALLIDAS ---")
    for ind_clave, ind_nombre, clave_estado, url in consultas_fallidas:
        print(f"Reintentando: {ind_clave} - Estado {clave_estado}")
        extraer_datos(ind_clave, ind_nombre, clave_estado, url)
        time.sleep(0.5)

# Procesamiento Inteligente de Fechas
if resultados_totales:
    df_export = pd.DataFrame(resultados_totales)
    
    # 1. Limpieza básica
    df_export['OBS_VALUE'] = pd.to_numeric(df_export['OBS_VALUE'], errors='coerce')
    
    # 2. Extracción del AÑO para filtrado dinámico
    # INEGI envía trimestre como "2024/01", tomamos los primeros 4 caracteres
    df_export['YEAR_NUM'] = df_export['TIME_PERIOD'].astype(str).str[:4].astype(int)
    
    # 3. Determinamos el año más reciente en toda la base (El "Presente")
    max_year = df_export['YEAR_NUM'].max()
    print(f"\n📅 Año más reciente detectado en los datos: {max_year}")
    print(f"📅 Se conservarán datos desde: {max_year - 1}")
    
    # 4. Filtro: Mantener solo (Año Actual) y (Año Anterior)
    df_final_export = df_export[df_export['YEAR_NUM'] >= (max_year - 1)].copy()
    
    # 5. Ordenamiento final y limpieza de columnas auxiliares
    df_final_export = df_final_export.sort_values(by=['CLAVE_INDICADOR', 'CVE_ENT', 'TIME_PERIOD'])
    df_final_export = df_final_export.drop(columns=['YEAR_NUM']).reset_index(drop=True)
    
    print("\n--- RESUMEN FINAL ---")
    print(f"Total de registros filtrados (Últimos 2 años móviles): {len(df_final_export)}")
    print(df_final_export.head(10))
    
    # Verificación de periodos únicos obtenidos
    print("\nPeriodos incluidos en el reporte final:")
    print(df_final_export['TIME_PERIOD'].unique())
    
else:
    print("\nNo se encontraron datos.")

--- INICIANDO EXTRACCIÓN HISTÓRICA MASIVA DE EXPORTACIONES ---
Consultando: 629659 - Total
Consultando: 696790 - Agricultura
Consultando: 696791 - Cría y explotación de animales
Consultando: 697788 - Pesca, caza y captura
  ⚠️ Fallo de red: ConnectTimeout (Intento 1/3)
Consultando: 629660 - Extracción de petroleo y gas
Consultando: 629661 - Minería de minerales (excepto petroleo y gas)
Consultando: 629662 - Alimentos
  ⚠️ Fallo de red: ConnectTimeout (Intento 1/3)
Consultando: 629663 - Bebidas y tabaco
  ⚠️ Fallo de red: ConnectTimeout (Intento 1/3)
Consultando: 629664 - Insumos y acabados textiles
Consultando: 629665 - Productos textiles (excepto prendas de vestir)
  ⚠️ Fallo de red: ConnectTimeout (Intento 1/3)
  ⚠️ Fallo de red: ConnectTimeout (Intento 1/3)
Consultando: 629666 - Prendas de vestir
Consultando: 629667 - Curtido, acabado y productos de cuero y piel
  ⚠️ Fallo de red: ConnectionError (Intento 1/3)
Consultando: 629668 - Madera
Consultando: 629669 - Papel
Consultando: 629